In [1]:
import sys
sys.path.append(r"../")

from src.data.data_preprocessing import transform_combined_to_classified

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
batch = tokenizer.encode('Today we have shitty weather', return_tensors='pt')

toxic = model(batch).logits.squeeze().tolist()[0] <= 0
print(toxic)

True


In [5]:
df = pd.read_csv('../data/interim/combined.tsv', sep='\t', header=0)
df = transform_combined_to_classified(df)
df.head()

Processing:   0%|          | 0/1262462 [00:00<?, ?it/s]

,text,toxic
0,alkar flood mental wast would explain high lev...,1
1,your becom disgust,1
2,well spare life,1
3,monkey wake,1
4,order kill,1


In [6]:
eval_ratio = 0.01

_, eval_df = train_test_split(df, test_size=eval_ratio, random_state=42, stratify=df['toxic'])

In [7]:
eval_df.values[:, 0]

array(['went one bedroom door open saw crazi woman bathtub',
       'even gon na get mexico honestli', 'your pain', ..., 'good game',
       'get water die',
       'wasnt call catfish catfish like ugliest fish sea'], dtype=object)

In [8]:
eval_dataloader = torch.utils.data.DataLoader(eval_df.values[:, 0], batch_size=32)

outputs = []
for text in tqdm(eval_df.values[:, 0]):
    batch = tokenizer.encode(text, return_tensors='pt')
    toxic = model(batch).logits.squeeze().tolist()[0] <= 0
    outputs.append(toxic)

  0%|          | 0/12590 [00:00<?, ?it/s]

In [ ]:
print(classification_report(eval_df['toxic'], outputs))

              precision    recall  f1-score   support

           0       0.68      0.89      0.77      6278
           1       0.85      0.58      0.69      6312

    accuracy                           0.73     12590
   macro avg       0.76      0.74      0.73     12590
weighted avg       0.76      0.73      0.73     12590



In [ ]:
incorrects = 0
for i in range(len(outputs)):
    if outputs[i] != eval_df['toxic'].values[i]:
        print(eval_df.values[i, 0] + ':', 'toxic' if outputs[i] == 1 else 'neutral')
        incorrects += 1
        if incorrects == 10:
            break

went one bedroom door open saw crazi woman bathtub: neutral
your pain: neutral
know sound ridicul son know differ cattl men confirm: neutral
guy yakuza gangster wont let one room: neutral
she sick: neutral
louie gon na go boob: neutral
dont need shot shoulder keep honor: neutral
lot peopl think mad russian: toxic
your egocentr miss therapi think term affect: neutral
hatr like eye: toxic
